# <center>DP

In [1]:
public class TreeNode {
    int val;
    TreeNode left;
    TreeNode right;
    TreeNode(int x) { val = x; }
}

## <font color='dddd00'>337. House Robber III (midium)</font> [Tree and DP]

The thief has found himself a new place for his thievery again. There is only one entrance to this area, called the "root." Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that "all houses in this place forms a binary tree". It will automatically contact the police if two directly-linked houses were broken into on the same night.

Determine the maximum amount of money the thief can rob tonight without alerting the police.

<font color='dd0000'>取得最大金额的情况为：
    
1. 不选根节点：***opt(root) = opt(root.left) + opt(root.right)***


2. 选根节点：  ***opt(root) = root + opt(left.left) + opt(left.right) + opt(right.left) + opt(right.right)***

根据上述递推式可构造出递归
</font>

In [2]:
// 解法1：递归 重复计算重叠子问题 复杂度高

public int rob(TreeNode root) {
    if(root == null) return 0;
    int a = root.val;
    if(root.left != null) a += rob(root.left.left) + rob(root.left.right);
    if(root.right != null) a += rob(root.right.left) + rob(root.right.right);
    int b = rob(root.left) + rob(root.right);
    return Math.max(a, b);
}

In [4]:
// 解法2：使用动态规划，自底向上，保存每一步的结果，最后推出根节点的值
//        要保存每个结点的结果，所以需要一个HashMap；DP要自底向上，所以需要先遍历树，从叶子结点算起

// 方法1：先先序遍历一遍树，把结点保存在栈中，同时初始化HashMap，再遍历一遍栈，实现自底向上
public int rob21(TreeNode root) {
    Stack<TreeNode> s = new Stack<>();
    Map<TreeNode, Integer> opt = new HashMap<>();
    preorder(root, s, opt);
    opt.put(null, 0);     // 叶子节点计算时会去找null值，所以放入表中

    while(!s.isEmpty()){
        TreeNode cur = s.pop();
        int a = opt.get(cur.left) + opt.get(cur.right);  // 不选
        int b = cur.val;
        if(cur.left != null) b += opt.get(cur.left.left) + opt.get(cur.left.right);
        if(cur.right != null) b += opt.get(cur.right.left) + opt.get(cur.right.right);  // 选
        opt.put(cur, Math.max(a, b));
    }
    return opt.get(root);
}

private void preorder(TreeNode x, Stack<TreeNode> s, Map<TreeNode, Integer> opt){
    if(x == null) return;
    s.push(x);
    opt.put(x, 0);
    preorder(x.left, s, opt);
    preorder(x.right, s, opt);
}


// 方法2：没有必要先遍历树，直接通过递归，在递归过程中把结果逐渐放入哈希表
//        动态规划本身是不需要递归的，因为这是一个树结构，所以递归是为了对树自底向上的查找
public int rob22(TreeNode root){
    Map<TreeNode, Integer> opt = new HashMap<>();
    return helper(root, opt);
}

private int helper(TreeNode root, Map<TreeNode, Integer> opt){
    if(root == null) return 0;
    if(opt.containsKey(root)) return opt.get(root);  // 先序遍历，根结点是最后处理的，当表里已经有root了，说明已经遍历完成，返回它的结果
    
    int a = root.val;
    if(root.left != null) a += helper(root.left.left, opt) + helper(root.left.right, opt);
    if(root.right != null) a += helper(root.right.left, opt) + helper(root.right.right,opt);  // 选
    
    int b = helper(root.left, opt) + helper(root.right, opt);  // 不选
    
    int max = Math.max(a, b);
    opt.put(root, max);      // 在表中记录当前结果
    return max;      // 返回值是每个结点能取到的最大值
}

<font color='dd0000'>从另一个角度考虑，每个结点有选/不选两个状态，每个结点能取的最大值的情况为：

1. 选：两个子节点不能选，那么最大值为 "**不选左子结点时，**左子树能取到的最大值 + **不选右子节点时，**右子节点能取到的最大值 + 自己本身的值"


2. 不选：最大值为 "**选左子节点时，**左子树的最大值 + **选右子节点时，**右子树的最大值"

所以，可以用一个大小为2的数组，分别保存每个节点选或不选两个状态的最大值

这样就不必考虑到孙子节点
</font>

In [8]:
// 解法3：res[0]保存不选该节点时的最大值，res[1]表示选该节点时的最大值

public int rob3(TreeNode root){
    int a = solve(root)[0];
    int b = solve(root)[1];
    return Math.max(a, b);
}

public int[] solve(TreeNode root){
    if(root == null) return new int[2];
    
    int[] res = new int[2];
    
    res[0] = Math.max(solve(root.left)[0], solve(root.left)[1]) + Math.max(solve(root.right)[0], solve(root.right)[1]);
    res[1] = root.val + solve(root.left)[0] + solve(root.right)[0];
    
    return res;
}

## <font color='dddd00'>32. Longest Valid Parentheses (hard)

Given a string containing just the characters ```'('``` and ```')'```, find the length of the longest valid (well-formed) parentheses substring.

In [1]:
// 状态设计：dp[i]表示以第i个字符结尾的最长有效长度
// 有：dp[0]一定为0; 
//     若arr[i]=='(',则dp[i]=0
// 当arr[i]==')'时，如果【i-dp[i-1]-1】位置正好是'('，那么dp[i] = dp[i-1]+2+dp[i-dp[i - 1]-1 - 1];否则，dp[i] = 0
// 最后，最大长度就是dp中最大的数值

public int longestValidParentheses(String s) {
    if(s == null || s.length() == 0)
            return 0;
        
    char[] arr = s.toCharArray();
    int[] dp = new int[s.length()];
    // dp[i]表示以字符arr[i]结尾的最大长度
    dp[0] = 0;
    int res = 0;
    for(int i = 1; i < arr.length; i++){
        if(arr[i] == ')'){
            int pre = i - dp[i - 1] - 1;
            if(pre >= 0 && arr[pre] == '(')
                dp[i] = dp[i - 1] + 2 + ((pre - 1 >= 0) ? dp[pre - 1] : 0);
        }
        res = Math.max(res, dp[i]);
    }
    return res;
}

## 322. Coin Change (medium)

You are given coins of different denominations(面值) and a total amount of money *amount*. Write a function to compute the **fewest** number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return ```-1```.

<font color='dd0000'>

问题可以转换为：每步可以上楼梯的阶数为coins数组里的值，用最少的步数上到amount楼。

**状态设计**：```dp[i]``` 表示走到第i楼的最小步数，初始状态为```dp[0] = 0```

**状态转移方程**：在第```i```步，能上的层数有```coins[1]、coins[2]、……```，所以```dp[i] = dp[i - coins[j]] + 1(当前这一步)```，从```dp[i - coins[j]]```中选择最小的那个，就是```dp[i]```的最小值，即：```dp[i] = min{dp[i - coins[j]} + 1```。

**注意点**：因为更新```dp[i]```时是不断取最小的，所以初值应设一个大的值，因为最大就是走到```amount```，所以设为```amount+1```很合适(```dp[0]```除外)。并且，会存在某一步没有解的情况，那么```dp[i]```的值就会始终是```amount+1```，最后可通过这个值来判断最后一步有没有解。
</font>

In [1]:
public int coinChange(int[] coins, int amount) {
    int[] dp = new int[amount + 1]; // 从0~amount,所以amount+1位的数组
    for(int k = 1; k < dp.length; k++)
        dp[k] = amount + 1;         // 设初值
    for(int i = 1; i < dp.length; i++){
        for(int j = 0; j < coins.length; j++){ // 检查每一步能上的阶数
            if(i - coins[j] >= 0){
                dp[i] = Math.min(dp[i], dp[i - coins[j]] + 1); // 不断更新最小值,如果没有更好的解,就保持初值amount+1
            }
        }
    }
    return dp[amount] > amount ? -1 : dp[amount];  // 如果dp[i]>amount,说明无解,返回-1；否则返回代表最优解的dp[i]
}

<font color='dd0000'>解法二：**贪心策略 + dfs搜索 + 剪枝**</font>

<font color='dd0000'>应用贪心策略的暴力dfs搜索策略为：从最大面值开始，先选最多的最大面值，总额减去当前已选择面额，再选最多的次大面值，依次类推，直到最小的面值求出一个解或无解，然后回溯到每一个大的面值处，将选择数量-1，不断更新解。但是这样将遍历所有的可能组合，复杂度很高。

但其实观察搜索树可知：**当搜索到某一层，发现此时的解(已选的个数)已经大于当前最优解了，那么就可以不再往下继续搜索了，实现剪枝**
</font>

In [3]:
public int coinChange2(int[] coins, int amount) {
    Arrays.sort(coins); // 必须对数组排好序，才能按从大到小的面值依次搜索
    dfs(coins, coins.length - 1, amount, 0);
    return ans;
}

private int ans = Integer.MAX_VALUE; // 因为要更新最小值，所以解的初值设为一个大数
private void dfs(int[] coins, int index, int amount, int count){ // index表示当前选择的面值的索引；count表示当前已经选了的个数
    int coin = coins[index];
    if(index == 0){   // base case:搜索到最小面值
        if(amount % coin == 0){  // 如果剩余额度正好可以整除当前面值
            ans = Math.min(ans, count + amount / coin);  // 更新解
        }
    }
    else{                                                            //k表示每次选择当前面值的个数,从最多开始逐渐减到0;
        for(int k = amount / coin; k >= 0 && count + k < ans; k--){ //【count+k<ans】控制剪枝:
            dfs(coins, index - 1, amount - coin * k, count + k);   //只有【已选个数count+当前选的个数k】<【当前最优解】时才继续搜索
        }                                                         //继续搜索时,代表面值的索引-1,总额度-当前已选面额,总个数+当前已选个数
    }
}

## 279. Perfect Squares (medium)

Given a positive integer n, find the least number of perfect square numbers (for example, ```1, 4, 9, 16, ...```) which sum to n.

<font color='dd0000'>动态规划解法：将小于给定数的完全平方数都求出来放进一个数组，转换为类似上题的问题：用最小数量的数组里的数组成给定数</font>

In [4]:
public int numSquares(int n) {
    List<Integer> squares = new ArrayList<>();  // 求完全平方数组
    int x = 1;
    while(x * x <= n){
        squares.add(x * x);
        x++;
    }
        
    int[] dp = new int[n + 1];          // dp[]初始化
    for(int k = 1; k < dp.length; k++){
        dp[k] = n + 1;
    }
    for(int i = 1; i < dp.length; i++){
        for(int j = 0; j < squares.size(); j++){
            if(i - squares.get(j) >= 0){
                dp[i] = Math.min(dp[i], dp[i - squares.get(j)] + 1);
            }
        }
    }
    return dp[n] > n ? -1 : dp[n];
}

// 代码优化：思路相同，但是没有必要用一个数组来存这些完全平方数，可以直接通过内部的循环解决
public int numSquares2(int n) {
    int[] dp = new int[n + 1];
    for(int i = 1; i < dp.length; i++){
        dp[i] = i;  // 因为对每个i,最坏情况就是由i个1组成,所以为dp[i]赋初值为i
        for(int j = 1; j * j < i; j++){  // 用循环控制每一步都是小于i的完全平方数
            dp[i] = Math.min(dp[i], dp[i - j * j] + 1);
        }
    }
    return dp[n];  // dp[n]一定有解,无需判断
}

<font color='dd0000'>解法二：BFS。See Search:BFS</font>